In [1]:
import json, pymongo, os

In [2]:
folder_path = '../db_hts/temp/NEW_test_files/'
string_folder_path = '../db_hts/temp/NEW_test_string_dict/'

file_list = os.listdir(folder_path)

file_list

['1801.json',
 '9504.json',
 '9808.json',
 '9801.json',
 '3707.json',
 '9025.json',
 '2853.json',
 '9202.json',
 '8606.json',
 '4113.json',
 '2930.json',
 '4910.json',
 '5109.json',
 '4808.json',
 '6805.json',
 '2716.json',
 '2202.json',
 '8442.json',
 '3918.json',
 '8453.json',
 '8709.json',
 '7217.json',
 '9111.json',
 '4202.json',
 '7105.json',
 '8421.json',
 '7804.json',
 '3801.json',
 '9913.json',
 '2615.json',
 '8480.json',
 '8904.json',
 '8458.json',
 '0910.json',
 '0203.json',
 '9603.json',
 '9703.json',
 '8905.json',
 '1302.json',
 '2824.json',
 '5702.json',
 '5903.json',
 '9013.json',
 '6211.json',
 '0407.json',
 '0505.json',
 '5309.json',
 '4013.json',
 '0410.json',
 '4205.json',
 '2844.json',
 '3913.json',
 '2827.json',
 '5807.json',
 '9007.json',
 '7412.json',
 '5206.json',
 '8415.json',
 '8512.json',
 '9305.json',
 '9006.json',
 '2929.json',
 '4415.json',
 '3404.json',
 '9008.json',
 '8110.json',
 '6101.json',
 '8109.json',
 '7503.json',
 '2846.json',
 '2616.json',
 '4115

In [3]:
class Connection:

    def __init__(self, db_path: str):

        self.client = pymongo.MongoClient(db_path)
        self.db = self.client['hts']
        self.collection_records = self.db['hts_records']
        self.collection_string_dict = self.db['string_dict']
        

    def closeConnection(self):
        self.client.close()
    
    def closeAllConnections(self):
        admin_db = self.client.admin

        all_connections = admin_db.command('serverStatus')['connections']

        for conn_type in all_connections:
            if conn_type != 'mongosh':
                admin_db.command('closeConnections', connectionType=conn_type)

        # Close the MongoDB client
        self.client.close()


def addHTSRecord(record: list, connection: Connection):

    for file in file_list:

        with open(f'{folder_path}{file}') as f:

            data = json.load(f)  

            try:
                connection.collection_records.insert_one(data)
            except:
                print(f'Error creating record in db for file {file}')

In [4]:
connection = Connection('mongodb://127.0.0.1:27017')

connection.client
connection.db

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'hts')

In [5]:
connection.closeConnection()
connection.closeAllConnections()

InvalidOperation: Cannot use MongoClient after close